In [1]:
import pandas as pd
import numpy as np

import geopy
import pyproj
from pyproj import Proj, transform

from time import time


In [45]:
#To replace with the actual path of the files: 
path = r'C:\Users\xxxx'
##


files = {'cours': 'cours.csv',
        'eleves': 'eleves.csv',
        'etablissement': 'structures.csv',
        'annuaire':'fr-en-annuaire-education2.csv',
        'stats':'StatEtablissement.csv',
        'capa':'capacites.xlsx',
        'dista':'distances_entre_etablissements.csv'}

In [3]:
dfCours = pd.read_csv(path+files['cours'])

In [4]:
dfEleves = pd.read_csv(path+files['eleves'])

C:\Users\perra\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (1,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
dfStruct = pd.read_csv(path+files['etablissement'], sep=';')
len(dfStruct.uaarne.unique())

dfCapa = pd.read_excel(path+files['capa'], 'Etablissement')
dfStats = pd.read_csv(path+files['stats'], header=None)

In [91]:
dfDista = pd.read_csv(path+files['dista'])
#dfDista.set_index('Identifiant_de_l_etablissement', inplace=True)

In [6]:
dfStats.columns=['uaarne',  'effectifs']
dfEtabTot = pd.merge(dfStats, dfCapa, left_on='uaarne', right_on='uairne')
dfEtabTot = dfEtabTot[['uaarne','effectifs', 'captot', 'denom', 'patron']]
dfEtabTot['%rempli']=dfEtabTot['effectifs']*100/dfEtabTot['captot']

In [7]:
dfEtabs = pd.read_csv(path+files['annuaire'], sep=';', error_bad_lines=False)

C:\Users\perra\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (8,10,60) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
print(len(dfEleves['Identifiant_division'].unique()))

dfEleves.columns

3614


Index(['ID', 'commune_residence', 'X', 'Y', 'SEXE', 'Annee_naissance',
       'PCS_resp1', 'Regime_scolaire', 'Etablissement_en_cours',
       'Code_nature_etablissement', 'Secteur_annee_en_cours',
       'Identifiant_division', 'Contrat_division', 'MEF_annee_en_cours',
       'Type_MEF_annee_en_cours', 'MEFSTAT11', 'Option1_annee_en_cours',
       'Option2_annee_en_cours', 'Option3_annee_en_cours',
       'Option4_annee_en_cours', 'Option5_annee_en_cours',
       'Option6_annee_en_cours', 'Option7_annee_en_cours',
       'Option8_annee_en_cours', 'Option9_annee_en_cours',
       'Option10_annee_en_cours', 'Option11_annee_en_cours',
       'Option12_annee_en_cours', 'Modalite_election_option1',
       'Modalite_election_option2', 'Modalite_election_option3',
       'Modalite_election_option4', 'Modalite_election_option5',
       'Modalite_election_option6', 'Modalite_election_option7',
       'Modalite_election_option8', 'Modalite_election_option9',
       'Modalite_election_option10',

In [9]:
listUnikStruct = dfStruct.uaarne.unique()
len(listUnikStruct)

qtyClasses = 0
for etab in listUnikStruct:
    dfSelected = dfEleves[dfEleves['Etablissement_en_cours']==etab]
    qtyClasses += len(dfSelected.Identifiant_division.unique())
    
print(qtyClasses)

11109


In [10]:
listUnikEtabs = list(dfEtabs.Identifiant_de_l_etablissement)
len(listUnikEtabs)

65609

In [11]:
listOK = []
for elem in listUnikStruct:
    if elem in listUnikEtabs:
        listOK+=[elem]

In [12]:
dfMerged = pd.merge(dfEleves, dfEtabs, left_on='Etablissement_en_cours', right_on='Identifiant_de_l_etablissement')

In [13]:
dfDistElevEtab = dfMerged[['ID', 'Regime_scolaire', 'Identifiant_division', 'Etablissement_en_cours', 'MEFSTAT11' ,'X', 'Y', 'coordonnee_X', 'coordonnee_Y']]

In [14]:
#dfDistElevEtab['CoordEleve']=(dfDistElevEtab['X'].map(str)+str(',')+dfDistElevEtab['Y'].map(str))
#dfDistElevEtab['CoordEtab']=dfDistElevEtab(dfDistElevEtab['coordonnee_X'].map(str)+str(',')+dfDistElevEtab['coordonnee_Y'].map(str))

In [20]:
dfDistNadrop = dfDistElevEtab.dropna()

In [21]:
dfDistNadrop['distanceX']=abs(dfDistNadrop['X']-dfDistNadrop['coordonnee_X'])
dfDistNadrop['distanceY']=abs(dfDistNadrop['Y']-dfDistNadrop['coordonnee_Y'])

C:\Users\perra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\perra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
dfDistNadrop['DistAbs']=np.sqrt(dfDistNadrop['distanceX']**2+dfDistNadrop['distanceY']**2)

C:\Users\perra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
listRegimes=[0,1,2,6]
dfNoInt = dfDistNadrop[dfDistNadrop['Regime_scolaire'].isin(listRegimes)]

In [24]:
dfCleaned = dfNoInt[dfNoInt['DistAbs']<100000]

mefList = ['21','22','23','24','43','51']
dfCleaned['firstMef']=dfCleaned['MEFSTAT11'].astype(str).str[0:2]
dfNew = dfCleaned[dfCleaned['firstMef'].isin(mefList)==True]

dfNew['firstMef']=dfNew['MEFSTAT11'].astype(str).str[0:3]
dfWork = dfNew[dfNew['firstMef']!=433]

C:\Users\perra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\perra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [25]:
dfWork = dfWork.drop(['firstMef'], axis=1)
len(dfWork.index)
dfClasses = dfStruct[dfStruct['elstty']=='D']

In [26]:
dfStr = dfStruct[['uaarne', 'mefsta11']].drop_duplicates()

In [27]:
StructLoc=pd.merge(dfStr, dfEtabs, left_on='uaarne', right_on='Identifiant_de_l_etablissement')
StructLoc = StructLoc[['uaarne', 'mefsta11', 'coordonnee_X', 'coordonnee_Y']]

In [110]:
def GetListOfPlaces():
    places = StructLoc.uaarne.unique()
    places = [elem for elem in places]
    return places

def FindClasses(etablissement):
    dfEstab = dfClasses[dfClasses['uaarne']==etablissement]
    return dfEstab
    

def RemoveClass(etablissement, mefs=None, classe=None):
    dfStudentsNeedToMove = dfWork[dfWork['Etablissement_en_cours']==etablissement]
    
    if mefs==None:
        mefs=list(set([elem for elem in dfClasses['mefsta11'][dfClasses['uaarne']==etablissement]]))
        for mef in mefs[1:]:
            dfStudentsNeedToMove=dfStudentsNeedToMove.append(dfStudentsNeedToMove[dfStudentsNeedToMove['MEFSTAT11']==mef])
        
    elif len(mefs)==1:
        dfStudentsNeedToMove = dfStudentsNeedToMove[dfStudentsNeedToMove['MEFSTAT11']==mefs[0]]
            
    dfStudentsNeedToMove['NewDist']=0
    dfStudentsNeedToMove['NewPlace']=0
    
    if classe!=None:
        dfStudentsNeedToMove = dfStudentsNeedToMove[dfStudentsNeedToMove['Identifiant_division']==classe]
        
    Struct = StructLoc[StructLoc['mefsta11']==mefs[0]]
    if len(mefs)>1:
        for mef in mefs[1:]:
            Struct=Struct.append(StructLoc[StructLoc['mefsta11']==mef])

    Struct = Struct[Struct['uaarne']!=etablissement]
    
    for x in range(len(dfStudentsNeedToMove.index)):
        Struct['X']=dfStudentsNeedToMove['X'].iloc[x]
        Struct['Y']=dfStudentsNeedToMove['Y'].iloc[x]
        Struct['Dist']=np.sqrt(abs(Struct['X']-Struct['coordonnee_X'])**2+abs(Struct['Y']-Struct['coordonnee_Y'])**2)
        dfStudentsNeedToMove['NewDist'].iloc[x] = min(Struct['Dist'])
        dfStudentsNeedToMove['NewPlace'].iloc[x] = StructLoc['uaarne'].iloc[Struct['Dist'].idxmin(axis=0)]
        dfStudentsNeedToMove['DistanceDiff']=dfStudentsNeedToMove['NewDist']-dfStudentsNeedToMove['DistAbs']
        dfStudentsFound = dfStudentsNeedToMove[['ID', 'MEFSTAT11', 'DistanceDiff', 'NewPlace']]
        
    uniquePlaces = dfStudentsFound.NewPlace.unique()
    
    dfStudentsFound = pd.merge(dfStudentsFound, dfEtabTot, left_on='NewPlace', right_on='uaarne')
        
    print('There are', len(dfStudentsFound.index), 'students who will be moved from', dfEtabTot['denom'][dfEtabTot['uaarne']==etablissement].iloc[0],dfEtabTot['patron'][dfEtabTot['uaarne']==etablissement].iloc[0])
    print()
    for place in uniquePlaces:
        
        daDista = float(dfDista[place][dfDista['Identifiant_de_l_etablissement']==etablissement].values)
        
        dfStudPlace = dfStudentsFound[dfStudentsFound['NewPlace']==place]
        print(len(dfStudPlace.index), 'students will be moved to ', place, ':', dfEtabTot['denom'][dfEtabTot['uaarne']==place].values[0],dfEtabTot['patron'][dfEtabTot['uaarne']==place].values[0])
        print('This place is',"{0:.2f}".format(daDista/1000), 'km from the former one')
        print('There are already:', dfEtabTot['effectifs'][dfEtabTot['uaarne']==place].values[0],'/',dfEtabTot['captot'][dfEtabTot['uaarne']==place].values[0], 'in this', dfEtabTot['denom'][dfEtabTot['uaarne']==place].values[0])
        print('There will be:', len(dfStudPlace.index)+dfEtabTot['effectifs'][dfEtabTot['uaarne']==place].values[0],'/',dfEtabTot['captot'][dfEtabTot['uaarne']==place].values[0], 'after transfer')
        print()
        print('The average distance added is', "{0:.2f}".format(np.mean(dfStudPlace['DistanceDiff'])/1000), 'km')
        print('Max distance added:', "{0:.2f}".format(max(dfStudPlace['DistanceDiff'])/1000), 'km, Min distance added:', "{0:.2f}".format(min(dfStudPlace['DistanceDiff'])/1000), 'km')
        print('For a total of:', "{0:.2f}".format(sum(dfStudPlace['DistanceDiff'])/1000), 'km, for an extra CO2 emission of:', "{0:.2f}".format(sum(dfStudPlace['DistanceDiff'])*0.130/25000), 'kg')
        print()
        print()
        
    return dfStudentsFound

In [125]:
GetListOfPlaces()
#dfEtabTot
#dfDista

['0240001W',
 '0240004Z',
 '0240005A',
 '0240006B',
 '0240007C',
 '0240010F',
 '0240011G',
 '0240012H',
 '0240013J',
 '0240014K',
 '0240015L',
 '0240016M',
 '0240021T',
 '0240024W',
 '0240025X',
 '0240026Y',
 '0240028A',
 '0240029B',
 '0240030C',
 '0240032E',
 '0240035H',
 '0240037K',
 '0240039M',
 '0240040N',
 '0240043S',
 '0240044T',
 '0240045U',
 '0240047W',
 '0240048X',
 '0240050Z',
 '0240052B',
 '0240053C',
 '0240055E',
 '0240056F',
 '0240065R',
 '0240066S',
 '0240073Z',
 '0240076C',
 '0240077D',
 '0240079F',
 '0240083K',
 '0240086N',
 '0240087P',
 '0240089S',
 '0240092V',
 '0240094X',
 '0240106K',
 '0240112S',
 '0240117X',
 '0240119Z',
 '0240121B',
 '0240158S',
 '0240650B',
 '0240924Z',
 '0240927C',
 '0240961P',
 '0240962R',
 '0240984P',
 '0240996C',
 '0240997D',
 '0241007P',
 '0241011U',
 '0241041B',
 '0241042C',
 '0241043D',
 '0241075N',
 '0241076P',
 '0241125T',
 '0241136E',
 '0241137F',
 '0330003Z',
 '0330008E',
 '0330010G',
 '0330011H',
 '0330017P',
 '0330018R',
 '0330020T',

In [112]:
FindClasses('0240021T')

In [113]:
#DfExtracted = RemoveClass('0240021T', ['23820031202'], '1 PRO')
DfExtracted = RemoveClass('0240021T', ['23820031202'], '1 PRO')

There are 9 students who will be moved from LYCEE POLYVALENT ALCIDE DUSOLIER

9 students will be moved to  0240028A : LYCEE PROFESSIONNEL PABLO PICASSO
This place is 38.11 km from the former one
There are already: 466 / 600 in this LYCEE PROFESSIONNEL
There will be: 475 / 600 after transfer

The average distance added is 27.62 km
Max distance added: 38.05 km, Min distance added: 3.15 km
For a total of: 248.58 km, for an extra CO2 emission of: 1.29 kg




In [114]:
DfExtracted = RemoveClass('0240021T')

There are 639 students who will be moved from LYCEE POLYVALENT ALCIDE DUSOLIER

429 students will be moved to  0240013J : LYCEE GENERAL GIRAUT DE BORNEIL
This place is 36.79 km from the former one
There are already: 175 / 700 in this LYCEE GENERAL
There will be: 604 / 700 after transfer

The average distance added is 24.94 km
Max distance added: 36.79 km, Min distance added: -17.87 km
For a total of: 10700.80 km, for an extra CO2 emission of: 55.64 kg


107 students will be moved to  0240032E : LYCEE GENERAL ET TECHNOLOGIQUE ARNAUT DANIEL
This place is 41.29 km from the former one
There are already: 488 / 1750 in this LYCEE GENERAL ET TECHNOLOGIQUE
There will be: 595 / 1750 after transfer

The average distance added is 15.51 km
Max distance added: 35.47 km, Min distance added: -23.42 km
For a total of: 1659.62 km, for an extra CO2 emission of: 8.63 kg


15 students will be moved to  0240026Y : LPO LYCEE DES METIERS ALBERT CLAVEILLE
This place is 38.33 km from the former one
There are a

In [115]:
#listOfTrucs = ['0240021T', '0240013J', '0240032E', '0240026Y', '0240028A']

In [123]:
#for etab in listOfTrucs:
#    thaDF = dfEtabs[dfEtabs['Identifiant_de_l_etablissement']==etab]
#    print(thaDF.Type_etablissement.values[0],thaDF.Nom_etablissement.values[0],thaDF.Adresse_1.values[0],thaDF.Adresse_3.values[0])

In [124]:
#dfEtabs